In [1]:
import pandas as pd
import json

"""
Construct data frame:
"""

df_columns = [
    'text',
    'likes'
]
df = {column: [] for column in df_columns}


"""
Read json with responses:
"""

with open('./Spiders/ithappens.json') as f:
    for n, line in enumerate(f):
        
        record = json.loads(line)
        
        df['text'].append(record['text'])
        df['likes'].append(record['likes'])
        
        if n % 1000 == 0:
            print "Responses have been read:", n

df = pd.DataFrame(df).ix[:, df_columns]

Responses have been read: 0
Responses have been read: 1000
Responses have been read: 2000
Responses have been read: 3000
Responses have been read: 4000
Responses have been read: 5000
Responses have been read: 6000
Responses have been read: 7000
Responses have been read: 8000
Responses have been read: 9000
Responses have been read: 10000


In [2]:
#df2 = df.sort(['likes'], ascending=False)
df2 = df.drop_duplicates(['text'])

In [3]:
df2.head(3)

,text,likes
0,<p>Работаю админом в бюджетной организации. По...,0
1,<p>Мне тут вспоминилось.<br>Пару лет назад раб...,0
2,<p>Взяли к нам на работу юрисконсульта.</p>\n<...,1


In [15]:
len(df2)

9898

In [12]:
len(df)

10893

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from random import shuffle

frac = 0.2
n = len(df2['text'])
texts = list(df2['text'])
likes = list(df2['likes'])

texts_and_likes = zip(texts, likes)
shuffle(texts_and_likes)
texts, likes = zip(*texts_and_likes)

X_train = texts[:-int(frac * n)]
y_train = likes[:-int(frac * n)]

X_test = texts[-int(frac * n):]
y_test = likes[-int(frac * n):]

In [ ]:
%%time
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(100) #LinearRegression()
vectorizer = CountVectorizer()
X_train_v = vectorizer.fit_transform(X_train)
X_test_v = vectorizer.transform(X_test)
model.fit(X_train_v.toarray(), y_train)

In [27]:
preds = model.predict(X_test_v.toarray())

In [28]:
print preds[:10]

[  747.96038344  1275.95441574   584.43729898  1340.09288958   956.28263211
  1355.12472132  1483.03374754  1229.5509253    984.8264811   1496.17364714]


In [29]:
print y_test[:10]

(1220, 1009, 802, 1360, 1482, 1020, 1886, 485, 1071, 1090)


In [35]:
top_pred_texts = [pred for pred, text in sorted(zip(preds, X_test), reverse=True)[:10]]
top_ans_texts = [ans for ans, text in sorted(zip(y_test, X_test), reverse=True)[:10]]

In [37]:
print "\n --------- \n".join(map(str, top_pred_texts))

5310.0505319
 --------- 
4350.2865391
 --------- 
4231.58739855
 --------- 
3947.05322861
 --------- 
3933.25336248
 --------- 
3854.88760027
 --------- 
3716.74761898
 --------- 
3643.94407351
 --------- 
3635.01037085
 --------- 
3458.72972733


In [38]:
print "\n --------- \n".join(map(str, top_ans_texts))

7079
 --------- 
6507
 --------- 
5217
 --------- 
5114
 --------- 
5057
 --------- 
5041
 --------- 
4912
 --------- 
4833
 --------- 
4824
 --------- 
4338


In [39]:
from sklearn.metrics import mean_squared_error

print mean_squared_error(preds, y_test) ** 0.5

799.608652085


In [42]:
z = "Однажды в девяностых я подрабатывал в конторе. Короче всё"

print model.predict(vectorizer.transform([z]))[0]

1437.43150353
